In [86]:
import pandas as pd
from functools import reduce

## Get data from Big Query

In [48]:
%%bigquery df
SELECT
    id
    ,symbol
    ,name
    ,date
    ,market_data.current_price as current_price
    ,market_data.market_cap as market_cap
    ,market_data.total_volume as total_volume  
FROM
    `crispy-computing-machine.crispy_dwh.crypto_btc`

Downloading: 100%|██████████| 1909/1909 [00:00<00:00, 2265.38rows/s]


Create a column for time ordered by day

In [49]:
df["date"]=pd.to_datetime(df["date"])
df.sort_values(by=["date"],ascending=True,inplace=True)
df.reset_index(drop=True,inplace=True)
df.reset_index(drop=False,inplace=True)
df.rename(columns={"index":"t"},inplace=True)

Summary of our dataframe

In [50]:
completitud=pd.DataFrame((1-df.isnull().sum()/df.shape[0])*100).reset_index().rename(columns={"index":"columna",0:"completitud"})
missings=pd.DataFrame(df.isnull().sum()).reset_index().rename(columns={"index":"columna",0:"missings"})
completitud=completitud.merge(missings,how="inner",on="columna")
tipo=pd.DataFrame(df.dtypes).reset_index().rename(columns={"index":"columna",0:"type"})
completitud=completitud.merge(tipo,how="inner",on="columna")
completitud

,columna,completitud,missings,type
0,t,100.000000,0,int64
1,id,100.000000,0,object
2,symbol,100.000000,0,object
3,name,100.000000,0,object
4,date,100.000000,0,datetime64[ns]
5,current_price,100.000000,0,float64
6,market_cap,99.947617,1,float64
7,total_volume,100.000000,0,float64


## Feature Engineering 

In [95]:
class window_time(object):
    def __init__(self,vobs,vdes,step,incremental):
        self.vobs = vobs
        self.vdes = vdes    
        self.step = step
        self.incremental = incremental
    
    def anclas(self, time_column):
        self.anclai = self.vobs-1
        self.anclaf = time_column.max()- self.vdes
        
    def ing_X(self,df,ancla,k):
        
        l = []
        
        aux = df.loc[(df['t'] <= ancla) & (df['t'] >= (ancla-k+1))]
        
        #current_price_mean
        piv = aux.pivot_table(index='symbol',columns='t',values='current_price',aggfunc='sum')
        piv[f'x_current_price_mean_{k}'] = piv.mean(axis=1)
        l.append(piv.filter(like='x_'))
        
        #current_price_std
        piv = aux.pivot_table(index='symbol',columns='t',values='current_price',aggfunc='sum')
        piv[f'x_current_price_std_{k}'] = piv.std(axis=1)
        l.append(piv.filter(like='x_'))   
        
        #market_cap_mean
        piv = aux.pivot_table(index='symbol',columns='t',values='market_cap',aggfunc='sum')
        piv[f'x_market_cap_mean_{k}'] = piv.mean(axis=1)
        l.append(piv.filter(like='x_'))

        #market_cap_std
        piv = aux.pivot_table(index='symbol',columns='t',values='market_cap',aggfunc='sum')
        piv[f'x_market_cap_std_{k}'] = piv.std(axis=1)
        l.append(piv.filter(like='x_'))   

        #total_volume_mean
        piv = aux.pivot_table(index='symbol',columns='t',values='total_volume',aggfunc='sum')
        piv[f'x_total_volume_mean_{k}'] = piv.mean(axis=1)
        l.append(piv.filter(like='x_'))

        #total_volume_std
        piv = aux.pivot_table(index='symbol',columns='t',values='total_volume',aggfunc='sum')
        piv[f'x_total_volume_std_{k}'] = piv.std(axis=1)
        l.append(piv.filter(like='x_'))   

        aux = reduce(lambda x,y:pd.merge(x,y,left_index=True,right_index=True,how='outer'),l).reset_index()
        aux.insert(1,'ancla',ancla)
    
        return aux
    
    def eng_X(self,df,um):
        
        cruzar = lambda x,y:pd.merge(x,y,on=um,how='outer')
        
        apilar = lambda x,y:x.append(y,ignore_index=True)
        
        X = reduce(apilar,
                   map(lambda ancla:
                       reduce(cruzar,
                              map(lambda k:self.ing_X(df,ancla,k),range(self.step,self.vobs+self.step,self.step))
                             ),
                   range(self.anclai,self.anclaf+1)
                      )
                  )
        return X

In [96]:
window=window_time(vobs=360,vdes=7,step=30,incremental=0.1)
window.anclas(df['t'])
X=window.eng_X(df,['symbol','ancla'])

In [98]:
X

t,symbol,ancla,x_current_price_mean_30,x_current_price_std_30,x_market_cap_mean_30,x_market_cap_std_30,x_total_volume_mean_30,x_total_volume_std_30,x_current_price_mean_60,x_current_price_std_60,...,x_market_cap_mean_330,x_market_cap_std_330,x_total_volume_mean_330,x_total_volume_std_330,x_current_price_mean_360,x_current_price_std_360,x_market_cap_mean_360,x_market_cap_std_360,x_total_volume_mean_360,x_total_volume_std_360
0,btc,359,14941.148400,3117.610279,2.500924e+11,5.233630e+10,6.592696e+09,4.485846e+09,11117.868008,4475.334927,...,6.860803e+10,6.734651e+10,1.279269e+09,2.304603e+09,3869.521263,3940.856061,6.410991e+10,6.618029e+10,1.405043e+09,2.431030e+09
1,btc,360,15159.204339,2973.328579,2.537686e+11,4.991503e+10,6.636728e+09,4.462180e+09,11295.180901,4469.172920,...,6.939257e+10,6.822961e+10,1.293963e+09,2.312517e+09,3912.229683,3992.622517,6.482790e+10,6.705373e+10,1.409154e+09,2.435779e+09
2,btc,361,15319.565408,2825.900664,2.564786e+11,4.743231e+10,6.697177e+09,4.429731e+09,11444.253397,4442.840687,...,7.010998e+10,6.890696e+10,1.309641e+09,2.321868e+09,3951.253852,4032.722185,6.548423e+10,6.773121e+10,1.411436e+09,2.439015e+09
3,btc,362,15493.587646,2633.297166,2.594171e+11,4.419091e+10,6.480330e+09,4.408564e+09,11594.122909,4411.212027,...,7.082786e+10,6.957521e+10,1.320731e+09,2.324779e+09,3990.312263,4072.302569,6.614124e+10,6.840010e+10,1.409641e+09,2.437046e+09
4,btc,363,15600.726784,2480.102459,2.612352e+11,4.159728e+10,6.352214e+09,4.435360e+09,11714.984746,4364.712778,...,7.147045e+10,7.004856e+10,1.331222e+09,2.327237e+09,4025.005353,4100.974046,6.672529e+10,6.888544e+10,1.395461e+09,2.409169e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,btc,1897,40403.765818,2298.532625,7.665440e+11,4.347525e+10,2.142000e+10,6.313578e+09,40308.990055,2649.705764,...,8.676774e+11,1.709480e+11,3.596856e+10,1.721069e+10,47066.695998,9316.786668,8.856287e+11,1.748073e+11,3.781927e+10,1.780282e+10
1539,btc,1898,40250.670364,2310.782617,7.637054e+11,4.374430e+10,2.112609e+10,6.428245e+09,40206.834185,2623.729611,...,8.664439e+11,1.705494e+11,3.581902e+10,1.718242e+10,47010.946319,9311.693011,8.846217e+11,1.747225e+11,3.769725e+10,1.781198e+10
1540,btc,1899,40164.493220,2281.488759,7.620990e+11,4.326496e+10,2.131846e+10,6.339182e+09,40157.851082,2605.505091,...,8.655279e+11,1.703527e+11,3.557413e+10,1.680718e+10,46959.352300,9300.734170,8.836901e+11,1.745358e+11,3.761198e+10,1.781705e+10
1541,btc,1900,40067.293573,2251.590152,7.602851e+11,4.266843e+10,2.160588e+10,6.099942e+09,40094.700949,2578.247887,...,8.646325e+11,1.701928e+11,3.543659e+10,1.674174e+10,46906.451433,9289.770404,8.827390e+11,1.743411e+11,3.751815e+10,1.781655e+10


In [97]:
1+1

2